In [50]:
import os, subprocess, re

In [58]:
def runNSave(cmd, path, get_times=True):
    # result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [ b.decode('UTF-8') for b in process.communicate() ]
    out = out.rstrip()
    # print("out:", out)
    print("error:", error)
        
    with open(path, 'w') as fh:
        fh.write(out)
    
    if get_times:
        netw_time = int(re.search("networking \d+ \[msec cputime\] (\d+) \[msec walltime\]", error).group(1))
        reas_time = int(re.search("reasoning \d+ \[msec cputime\] (\d+) \[msec walltime\]", error).group(1))
        return netw_time, reas_time

def record(times_file, query, data, type, phase, netw_time, reas_time):
    times_file.write(f"{query},{data},{type},{phase},{netw_time},{reas_time}\n")

In [55]:
# path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/"
path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix/"
for file in os.listdir(path):    
    if not file.endswith(".sparql"):
        continue
    
    file = file[:file.index(".")] + ".n3"
    or_file = os.path.join(path, "n3", file)
    norm_file = os.path.join(path, "n3", "normalized", file)
    
    print(file)
    
    # ground
    runNSave(["eye", "--quiet", or_file, "--no-qvars", "--nope", "--pass-all"], norm_file, get_times=False)
    
    # normalize
    runNSave(["eye", "--quiet", norm_file, "aux_list.n3", "--query", "list-predicate.n3", "--quantify", "http://www.w3.org/2000/10/swap/var#", "--nope"], norm_file, get_times=False)

query-24.n3
query-47.n3
query-36.n3
query-28.n3
query-12.n3
query-1.n3
query-34.n3
query-49.n3
query-38.n3
query-45.n3
query-26.n3
query-3.n3
query-10.n3
query-18.n3
query-7.n3
query-14.n3
query-41.n3
query-22.n3
query-30.n3
query-16.n3
query-5.n3
query-9.n3
query-32.n3
query-20.n3
query-43.n3
query-0.n3
query-13.n3
query-46.n3
query-25.n3
query-29.n3
query-37.n3
query-11.n3
query-2.n3
query-48.n3
query-35.n3
query-27.n3
query-44.n3
query-39.n3
query-23.n3
query-40.n3
query-31.n3
query-19.n3
query-15.n3
query-6.n3
query-33.n3
query-42.n3
query-21.n3
query-4.n3
query-17.n3
query-8.n3


In [60]:
# path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/"
path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix/"

times_file = open(os.path.join(path, "n3", "results", "times.csv"), 'w')
times_file.write("query,data,type,phase,netw_time,reas_time\n")

files = list(os.listdir(os.path.join(path, "n3", "normalized")))
files.sort()
for file in files:
    if not file.startswith("query") and not file.endswith(".n3"):
        continue
    
    norm_file = os.path.join(path, "n3", "normalized", file)
    data_file = "data.n3"
    data_path = os.path.join(path, data_file)
    dir_res_file = os.path.join(path, "n3", "results", "direct", file)
    bwd_res_file = os.path.join(path, "n3", "results", "bwd", file)
    fwd_res_file = os.path.join(path, "n3", "results", "fwd", file)
    tmp_file = "tmp.n3"
    
    print(file)
    
    # - direct 
    netw_time, reas_time = runNSave(["eye", data_path, "property-paths-direct.n3", "--query", norm_file, "--nope"], dir_res_file)
    record(times_file, file, data_file, 'direct', 'n/a', netw_time, reas_time)
    
    # - backward
    netw_time1, reas_time1 = runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation-backwards.n3", "--nope"], tmp_file)
    record(times_file, file, data_file, 'bwd', 'create', netw_time1, reas_time1)
    netw_time2, reas_time2 = runNSave(["eye", data_path, tmp_file, "--query", norm_file, "--nope"], bwd_res_file)
    record(times_file, file, data_file, 'bwd', 'run', netw_time2, reas_time2)
    record(times_file, file, data_file, 'bwd', 'total', netw_time1+netw_time2, reas_time1+reas_time2)
        
    # - forward
    netw_time1, reas_time1 = runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation.n3", "--nope"], tmp_file)
    record(times_file, file, data_file, 'fwd', 'create', netw_time1, reas_time1)
    netw_time2, reas_time2 = runNSave(["eye", data_path, tmp_file, "--query", norm_file, "--nope"], fwd_res_file)
    record(times_file, file, data_file, 'fwd', 'run', netw_time2, reas_time2)
    record(times_file, file, data_file, 'fwd', 'total', netw_time1+netw_time2, reas_time1+reas_time2)
    
    times_file.flush()

query-0.n3
error: eye --quiet /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix/data.n3 property-paths-direct.n3 --query /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix/n3/normalized/query-0.n3 --nope
EYE v10.24.10 (2024-10-02)
SWI-Prolog version 9.2.7
starting 29 [msec cputime] 32 [msec walltime]
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix/data.n3 SC=55
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/property-paths-direct.n3 SC=14
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix/n3/normalized/query-0.n3 SC=0
networking 7 [msec cputime] 8 [msec walltime]
reasoning 0 [msec cputime] 0 [msec walltime]
2025-05-20T14:30:09.797Z in=69 out=0 ent=0 step=0 brake=1 inf=121121 sec=0.036 inf/sec=3364472


error: eye --quiet /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix/n3/normalized/query-0.n3 rule-creation.n3 --query rule-creation-backwards.n3 --nope
EYE v10.24.10 (2024-10-02)
SWI-Prolog versi